We try to bootstrap Evolutionary algorithms. We try to run one EA(here, inner EA) within onother EA (Outer EA)

In [ ]:
import random
import numpy as np


In [ ]:
#InnerEA EA

In [ ]:
class Die:
    value = -1  # Class variable
    __name__ = "Die"
    def __init__(self):
        self.name = "Die"

    def roll(self):
        Die.value = random.randint(1, 6)
        return Die.value


In [ ]:
class innerIndividual:
    def __init__(self):
        self.diceValues = [random.randint(1, 6) for _ in range(100)]
        self.fitnessValue = 0

    def mutate(dice_values, mutation_rate):
        mutated_values = []
        for value in dice_values:
            if  random.random() < mutation_rate:
                mutated_value = value + int(np.random.normal(loc=0, scale=1))  # Mutate using normal distribution
                mutated_values.append(max(min(mutated_value, 6), 1))  # Ensure mutated value stays within range [1, 6]
            else:
                mutated_values.append(value)  # Keep the original value
        return mutated_values

    def fitness(self):
      return self.diceValues.count(6)


In [ ]:
def sort_individuals_by_fitness(fitness_scores, individuals):
    sorted_individuals = [ind for _, ind in sorted(zip(fitness_scores, individuals), key=lambda pair: pair[0], reverse=True)]
    return sorted_individuals

def mutate(dice_values, mutation_rate):
    mutated_values = []
    for value in dice_values:
        if value != 6 and random.random() < mutation_rate:
            mutated_value = value + int(np.random.normal(loc=0, scale=1))  # Mutate using normal distribution
            mutated_values.append(max(min(mutated_value, 6), 1))  # Ensure mutated value stays within range [1, 6]
        else:
            mutated_values.append(value)  # Keep the original value
    return mutated_values

def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1))  # Randomly select crossover point
    child_values = parent1[:crossover_point] + parent2[crossover_point:]
    return child_values

def uniformCrossover(parent1,parent2):
    child_values = []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child_values.append(parent1[i])
        else:
            child_values.append(parent2[i])

    return child_values

def evolution_dice_algorithm(**kwargs):
    population_size = kwargs.get("populationSize")
    selection_size = kwargs.get("selectionSize")
    mutation_rate = kwargs.get("mutationRate")
    generations = kwargs.get("generations")

    population_list = [innerIndividual() for _ in range(population_size)]
    for _ in range(generations):
        fitness_scores = [individual.fitness() for individual in population_list]
        population_list = sort_individuals_by_fitness(fitness_scores, population_list)
        selected_population = population_list[:selection_size]
        if  sum(fitness_scores) == 100 * len(fitness_scores):
          break

        new_population = selected_population[:]  # Include parents in the new population
        for _ in range(population_size - selection_size):  # Fill remaining population slots
            parent1, parent2 = random.sample(selected_population, 2)
            child1_values = uniformCrossover(parent1.diceValues, parent2.diceValues)
            child2_values = uniformCrossover(parent2.diceValues, parent1.diceValues)
            child1_values = mutate(child1_values, mutation_rate)
            child2_values = mutate(child2_values, mutation_rate)
            child1 = innerIndividual()
            child1.diceValues = child1_values
            child2 = innerIndividual()
            child2.diceValues = child2_values
            new_population.extend([child1, child2])

        population_list = new_population

    population_list = sort_individuals_by_fitness(fitness_scores, population_list)
    selected_population = population_list[:selection_size]
    #return [individual.diceValues for individual in selected_population]
    return selected_population[0].diceValues


Outer EA

In [ ]:
class OuterIndividual:
    def __init__(self, population_size_limit):
        self.population = random.randint(4, population_size_limit)
        self.selection = random.randint(2, self.population - 1)
        self.mutation = random.uniform(0, 1)
        self.generations = random.randint(1,10)
        self.fitnessValue = 0

    def setFitness(self, value):
        self.fitnessValue = value

    def printInfo(self):
        print(f"population: {self.population}, selection: {self.selection}")
        print(f"Mutation: {self.mutation}, generation: {self.generations}")
        print(f"Fitness: {self.fitnessValue}")


In [ ]:
def getFitnessList(outer_population_list):
    fitnessValues = []
    for outer_individuals in outer_population_list:
        krawgs = {
            "populationSize": outer_individuals.population,
            "selectionSize": outer_individuals.selection,
            "mutationRate": outer_individuals.mutation,
            "generations": outer_individuals.generations,
        }
        result= evolution_dice_algorithm(**krawgs)
        fitnessValues.append( (6 *len(result)) - (sum(result))) # the lower the fitness value the better the algorithm
    return fitnessValues


In [ ]:
def outer_crossover(parent1,parent2):
  child = OuterIndividual(0)
  child.population = random.choice([parent1.population, parent2.population])
  child.selection = random.choice([parent1.selection, parent2.selection])
  child.mutation = random.choice([parent1.mutation, parent2.mutation])
  child.generations = random.choice([parent1.generations, parent2.generations])
  return child

In [ ]:
def outer_mutate(child,mutation_rate):

  if random.random() < mutation_rate:
    child.population = max(4, child.population + random.randint(-1, 1))
  if random.random() < mutation_rate:
    child.selection = max(2, min(child.selection + random.randint(-1, 1),child.population-1))
  if random.random() < mutation_rate:
    child.mutation = max(0, min(1, child.mutation + random.uniform(-0.1, 0.1)))
  if random.random() < mutation_rate:
    child.generations = max(1, child.generations + random.randint(-1, 1))

  return child



In [ ]:
def outer_evolution_algorithm(**kwargs):
    outer_population_size = kwargs.get("populationSize")
    outer_selection_size = kwargs.get("selectionSize")
    outer_mutation_rate = kwargs.get("mutationRate")
    outer_generations = kwargs.get("generations")

    outer_population_list = [OuterIndividual(outer_population_size) for _ in range(outer_population_size)]
    fitness_value = getFitnessList(outer_population_list)
    for _ in range(outer_generations):
        fitness_values = getFitnessList(outer_population_list)
        for index, member in enumerate(outer_population_list):
            member.setFitness(fitness_values[index])

        outer_population_list = sorted(outer_population_list, key=lambda x: x.fitnessValue)
        new_population= outer_population_list[:outer_selection_size] #selecting best parents for furthur repopulation
        if all(member.fitnessValue == 0 for member in new_population): # Exit the loop if all selected individuals have fitness value 0
              break

        for _ in range(outer_population_size - outer_selection_size):
          parent1, parent2 = random.sample(new_population, 2)
          child = outer_crossover(parent1.diceValues, parent2.diceValues)
          child = outer_mutate(child,outer_mutation_rate)
          new_population.append(child)

        outer_population_size = new_population

    for index, member in enumerate(outer_population_list):
          member.setFitness(fitness_values[index])
    outer_population_list = sorted(outer_population_list, key=lambda x: x.fitnessValue)
    new_population= outer_population_list[:outer_selection_size] #selecting best parents for furthur repopulation

    return new_population




In [ ]:
# Corrected argument dictionary
arg = {
    "populationSize": 100,
    "selectionSize": 20,
    "mutationRate": 0.1,
    "generations": 1,  # Corrected key name
}

result = outer_evolution_algorithm(**arg)

In [ ]:
for res in result:
  res.printInfo()

population: 62, selection: 29
Mutation: 0.433179791027932, generation: 973
Fitness: 0
population: 93, selection: 59
Mutation: 0.8094062285851792, generation: 485
Fitness: 0
population: 75, selection: 30
Mutation: 0.4088520006889834, generation: 925
Fitness: 0
population: 99, selection: 85
Mutation: 0.17740756847588612, generation: 260
Fitness: 0
population: 89, selection: 31
Mutation: 0.16311946152101564, generation: 722
Fitness: 0
population: 30, selection: 6
Mutation: 0.5794124242650734, generation: 250
Fitness: 0
population: 50, selection: 11
Mutation: 0.7961670438145378, generation: 781
Fitness: 0
population: 57, selection: 7
Mutation: 0.7955840627681805, generation: 419
Fitness: 0
population: 80, selection: 19
Mutation: 0.9001110001975897, generation: 758
Fitness: 0
population: 91, selection: 17
Mutation: 0.26229969231856354, generation: 160
Fitness: 0
population: 66, selection: 20
Mutation: 0.3022874551021194, generation: 57
Fitness: 0
population: 83, selection: 7
Mutation: 0.924